In [1]:
from utils import base_configs, deps, tr_va_te_split
from utils import tr_va_te_split
from utils.helpers import dir_helpers, rw_csv_helpers, feature_distr_helpers, feature_transform_helpers

In [2]:
import sys, math
import import_ipynb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### 0 Data load

In [4]:
df_orig = rw_csv_helpers.read_csv_file("op/hints6_public_filtered_v1_cleaned_encoded.csv", verbose = 1)

Loaded: /home/ppanta/puru_proj/proj_v0/hints6_v0/op/hints6_public_filtered_v1_cleaned_encoded.csv
────────────────────────────────────────────────────────────────────────────────
Shape: (4865, 27)
────────────────────────────────────────────────────────────────────────────────
All columns: ['FreqGoProvider', 'Deaf', 'MedConditions_Diabetes', 'MedConditions_HighBP', 'MedConditions_HeartCondition', 'MedConditions_LungDisease', 'MedConditions_Depression', 'AverageSleepNight', 'AverageTimeSitting', 'EverHadCancer', 'Age', 'BirthGender', 'BMI', 'PHQ4', 'WeeklyMinutesModerateExercise', 'AvgDrinksPerWeek', 'GeneralHealth_Excellent', 'GeneralHealth_VeryGood', 'GeneralHealth_Good', 'GeneralHealth_Fair', 'GeneralHealth_Poor', 'smokeStat_Current', 'smokeStat_Former', 'smokeStat_Never', 'eCigUse_Current', 'eCigUse_Former', 'eCigUse_Never']


In [5]:
counts = feature_distr_helpers.count01(df_orig.copy(), "MedConditions_HeartCondition", verbose=1)

Counts for column 'MedConditions_HeartCondition' (only 0 and 1):
MedConditions_HeartCondition
0    4412
1     453
Name: count, dtype: int64
Total (0/1 only): 4865


### 1 Train - Validation - Test split

In [6]:
df_orig = df_orig.copy()
target_col = "MedConditions_HeartCondition"

X = df_orig.drop(columns=[target_col])
y = df_orig[target_col]

result = tr_va_te_split.data_preprocessing(
    verbose=0,       # 0, 1, or 2
    X=X, y=y,
    balance_method="adasyn",  # or 'smote', 'smoteenn', 'none'
    balance_kwargs={"n_neighbors": 5}  # optional
)

### 2 Train - Validation - Test value assignments

In [7]:
print_result = feature_distr_helpers.print_shapes_and_features(result, verbose = 1)

# Value assignments for further calculations
X_train_res_scaled = result['X_train_res_scaled']
X_val_scaled       = result['X_val_scaled']
X_test_scaled      = result['X_test_scaled']

X_train     = result['X_train']
X_train_res = result['X_train_res']
X_val       = result['X_val']
X_test      = result['X_test']

y_train     = result['y_train']
y_train_res = result['y_train_res']
y_val       = result['y_val']
y_test      = result['y_test']

features    = result['features']


X_train shape:            X = (2919, 26),      y = (2919,)
X_train_res shape:        X = (5271, 26),  y = (5271,)
X_val shape:              X = (973, 26),        y = (973,)
X_test shape:             X = (973, 26),       y = (973,)
X_train_res_scaled shape: X = (5271, 26)
X_val_scaled shape:       X = (973, 26)
X_test_scaled shape:      X = (973, 26)
features length:          n = 26


In [8]:
# ============================================================
# Multi-model SHAP + Composite Importance using YOUR splits
# (uses: X_train_res_scaled, X_val_scaled, X_test_scaled,
#        X_train, X_train_res, X_val, X_test,
#        y_train, y_train_res, y_val, y_test, features)
# ============================================================

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# XGBoost (optional)
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except Exception:
    HAS_XGB = False

import shap

RANDOM_STATE = 42
rng = np.random.default_rng(RANDOM_STATE)

# -----------------------------------------------------------
# 0) Coercion helpers
# -----------------------------------------------------------
def to_df(X, columns):
    if isinstance(X, pd.DataFrame):
        return X.copy()
    X = np.asarray(X)
    if columns is None:
        columns = [f"f{i}" for i in range(X.shape[1])]
    return pd.DataFrame(X, columns=columns)

def to_series(y, name="target"):
    if isinstance(y, pd.Series):
        return y.copy()
    return pd.Series(np.asarray(y).ravel(), name=name)

# -----------------------------------------------------------
# 1) Metrics
# -----------------------------------------------------------
def expected_calibration_error(y_true, y_prob, n_bins=15):
    bins = np.linspace(0, 1, n_bins + 1)
    inds = np.digitize(y_prob, bins) - 1
    ece = 0.0
    N = len(y_true)
    for b in range(n_bins):
        mask = inds == b
        if mask.sum() == 0:
            continue
        conf = y_prob[mask].mean()
        acc = y_true[mask].mean()
        ece += (mask.sum() / N) * abs(acc - conf)
    return float(ece)

def perf_summary(y_true, y_prob):
    return {
        "roc_auc": roc_auc_score(y_true, y_prob),
        "pr_auc": average_precision_score(y_true, y_prob),
        "ece": expected_calibration_error(y_true, y_prob, n_bins=15)
    }

# -----------------------------------------------------------
# 2) SHAP utilities (API-safe)
# -----------------------------------------------------------
def mean_abs_shap(shap_values, feature_names):
    vals = np.abs(np.asarray(shap_values)).mean(axis=0)
    return pd.Series(vals, index=feature_names)

def _pos_index_from_model(model):
    """Try to map label '1' to its index in model.classes_. Defaults to 1 or last class."""
    try:
        classes = getattr(model, "classes_", None)
        if classes is not None:
            classes = np.asarray(classes)
            if 1 in classes:
                return int(np.where(classes == 1)[0][0])
            return int(np.argmax(classes))
    except Exception:
        pass
    return 1

def tree_shap(model, X_bg, X_eval, feature_names, positive_class_index=None):
    """
    Robust Tree SHAP wrapper:
    - Older SHAP: returns list [class0, class1]
    - Newer SHAP: returns (n_samples, n_features) or (n_samples, n_features, n_classes)
    """
    if positive_class_index is None:
        positive_class_index = _pos_index_from_model(model)

    explainer = shap.TreeExplainer(model, data=X_bg)
    sv = explainer.shap_values(X_eval)

    # Older SHAP: list per class
    if isinstance(sv, list):
        if len(sv) > positive_class_index:
            sv = sv[positive_class_index]
        else:
            sv = sv[-1]

    sv = np.asarray(sv)
    if sv.ndim == 3:
        # (n_samples, n_features, n_classes)
        if sv.shape[-1] > positive_class_index:
            sv = sv[..., positive_class_index]
        else:
            sv = sv[..., -1]

    return mean_abs_shap(sv, feature_names)

def linear_shap_trained_lr_on_scaled(lr_estimator, X_bg_scaled, X_eval_scaled, feature_names):
    """
    SHAP >=0.41: LinearExplainer(model, masker=shap.maskers.Independent(X_bg))
    Older SHAP:  LinearExplainer(model, X_bg)
    """
    try:
        masker = shap.maskers.Independent(X_bg_scaled)
        explainer = shap.LinearExplainer(lr_estimator, masker=masker)
        sv = explainer.shap_values(X_eval_scaled)
    except TypeError:
        # Fallback for older SHAP versions
        explainer = shap.LinearExplainer(lr_estimator, X_bg_scaled)
        sv = explainer.shap_values(X_eval_scaled)
    return mean_abs_shap(sv, feature_names)

# -----------------------------------------------------------
# 3) Normalization + composite
# -----------------------------------------------------------
def normalize_importance(s, how="sum1"):
    x = s.clip(lower=0).astype(float)
    if how == "sum1":
        denom = x.sum()
        return (x / denom) if denom > 0 else x * 0.0
    elif how == "minmax":
        lo, hi = x.min(), x.max()
        return (x - lo) / (hi - lo) if hi > lo else x * 0.0
    else:
        raise ValueError("how must be 'sum1' or 'minmax'")

def _flatten_model_columns(df):
    """
    If concat created a MultiIndex on columns, drop to the top level (model names).
    If it's already a simple Index, leave it as-is.
    """
    if isinstance(df.columns, pd.MultiIndex):
        return df.copy().set_axis(df.columns.get_level_values(0), axis=1)
    return df

def composite_from_models(imp_dict, weights, scale="sum1"):
    # Align by feature index
    df = pd.concat(imp_dict, axis=1)
    df = _flatten_model_columns(df)  # <-- critical fix (no truncation of names)

    # Normalize per-model importances
    for m in df.columns:
        df[m] = normalize_importance(df[m], how=scale)

    # Normalize and align weights to columns safely
    w_raw = pd.Series(weights, dtype=float)
    w_norm = w_raw / w_raw.sum() if w_raw.sum() > 0 else pd.Series(1/len(w_raw), index=w_raw.index)
    w_aligned = w_norm.reindex(df.columns).fillna(0.0)

    comp = df.mul(w_aligned, axis=1).sum(axis=1)
    out = df.copy()
    out["CompositeImportance"] = comp
    out = out.sort_values("CompositeImportance", ascending=False)
    return out, w_aligned

# -----------------------------------------------------------
# 4) Sampling helpers for SHAP
# -----------------------------------------------------------
def sample_idx(n, k):
    return rng.choice(n, size=min(k, n), replace=False)

# ============================================================
# >>> Your splits (already defined in your session) <<<
# X_train_res_scaled, X_val_scaled, X_test_scaled
# X_train, X_train_res, X_val, X_test
# y_train, y_train_res, y_val, y_test
# features
# ============================================================

# Coerce shapes/columns
X_train_res_scaled = to_df(X_train_res_scaled, features)
X_val_scaled       = to_df(X_val_scaled,       features)
X_test_scaled      = to_df(X_test_scaled,      features)

X_train     = to_df(X_train,     features)
X_train_res = to_df(X_train_res, features)
X_val       = to_df(X_val,       features)
X_test      = to_df(X_test,      features)

y_train     = to_series(y_train)
y_train_res = to_series(y_train_res)
y_val       = to_series(y_val)
y_test      = to_series(y_test)

feature_names = list(X_train.columns)

# -----------------------------------------------------------
# 5) Train models
# -----------------------------------------------------------
models = {}

# Logistic Regression (scaled, resampled)
lr = LogisticRegression(max_iter=500, class_weight="balanced", random_state=RANDOM_STATE)
lr.fit(X_train_res_scaled, y_train_res)
models["lr"] = lr

# Random Forest (unscaled, resampled)
rf = RandomForestClassifier(
    n_estimators=600,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    class_weight="balanced",
    n_jobs=-1,
    random_state=RANDOM_STATE
)
rf.fit(X_train_res, y_train_res)
models["rf"] = rf

# XGBoost (optional, unscaled, resampled)
if HAS_XGB:
    xgb = XGBClassifier(
        n_estimators=700,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.9,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        objective="binary:logistic",
        eval_metric="logloss",
        random_state=RANDOM_STATE,
        n_jobs=-1,
        tree_method="hist"
    )
    xgb.fit(X_train_res, y_train_res)
    models["xgb"] = xgb

# -----------------------------------------------------------
# 6) Validation performance (for weights)
# -----------------------------------------------------------
def proba(model, X):
    return model.predict_proba(X)[:, 1]

perf = {
    "lr":  perf_summary(y_val.values, proba(models["lr"],  X_val_scaled)),
    "rf":  perf_summary(y_val.values, proba(models["rf"],  X_val)),
}
if "xgb" in models:
    perf["xgb"] = perf_summary(y_val.values, proba(models["xgb"], X_val))

perf_df = pd.DataFrame(perf).T
print("Validation performance (weights source):")
print(perf_df[["roc_auc", "pr_auc", "ece"]].round(4))

# -----------------------------------------------------------
# 7) SHAP computations (subsample for speed)
# -----------------------------------------------------------
bg_idx_lr = sample_idx(len(X_train_res_scaled), 200)
ev_idx_lr = sample_idx(len(X_val_scaled), 300)
bg_idx_tr = sample_idx(len(X_train_res), 200)
ev_idx_tr = sample_idx(len(X_val), 300)

imp = {}
# LR SHAP on scaled data
imp["lr"] = linear_shap_trained_lr_on_scaled(
    lr_estimator=models["lr"],
    X_bg_scaled=X_train_res_scaled.iloc[bg_idx_lr],
    X_eval_scaled=X_val_scaled.iloc[ev_idx_lr],
    feature_names=feature_names
)

# RF SHAP on unscaled data (robust to 3D outputs)
imp["rf"] = tree_shap(
    model=models["rf"],
    X_bg=X_train_res.iloc[bg_idx_tr],
    X_eval=X_val.iloc[ev_idx_tr],
    feature_names=feature_names
)

# XGB SHAP (if available)
if "xgb" in models:
    imp["xgb"] = tree_shap(
        model=models["xgb"],
        X_bg=X_train_res.iloc[bg_idx_tr],
        X_eval=X_val.iloc[ev_idx_tr],
        feature_names=feature_names
    )

imp_df = pd.DataFrame({m: s for m, s in imp.items()})
imp_df.index.name = "feature"
print("\nPer-model mean(|SHAP|) (unnormalized) — head:")
print(imp_df.head(10).round(6))

# -----------------------------------------------------------
# 8) Build model weights (AUC-only by default; include (1-ECE) by setting beta=1.0)
# -----------------------------------------------------------
alpha, beta = 1.0, 0.0  # alpha: AUC exponent; beta: (1 - ECE) exponent
weights = {}
for m, met in perf.items():
    auc = met["roc_auc"]
    ece = met["ece"]
    weights[m] = (auc ** alpha) * ((1 - ece) ** beta)

# -----------------------------------------------------------
# 9) Composite importance (robust column + weight alignment)
# -----------------------------------------------------------
comp_df, w_used = composite_from_models(imp, weights=weights, scale="sum1")
print("\nModel weights used (normalized, aligned to columns):")
print(w_used.round(4))

print("\nTop features by CompositeImportance:")
print(comp_df.head(20).round(5))

# -----------------------------------------------------------
# 10) Test metrics (final reporting)
# -----------------------------------------------------------
test_metrics = {
    "lr":  perf_summary(y_test.values, proba(models["lr"],  X_test_scaled)),
    "rf":  perf_summary(y_test.values, proba(models["rf"],  X_test)),
}
if "xgb" in models:
    test_metrics["xgb"] = perf_summary(y_test.values, proba(models["xgb"], X_test))

print("\nTest-set metrics:")
print(pd.DataFrame(test_metrics).T[["roc_auc", "pr_auc", "ece"]].round(4))

# -----------------------------------------------------------
# 11) Optional exports
# -----------------------------------------------------------
# comp_df.to_csv("composite_importance.csv")
# imp_df.to_csv("per_model_mean_abs_shap.csv")
# perf_df.to_csv("validation_metrics_for_weights.csv")
# pd.DataFrame(test_metrics).T.to_csv("test_metrics.csv")


Validation performance (weights source):
     roc_auc  pr_auc     ece
lr    0.7691  0.2594  0.0471
rf    0.7156  0.1941  0.0638
xgb   0.7012  0.1782  0.0745


100%|===================| 599/600 [01:28<00:00]        


Per-model mean(|SHAP|) (unnormalized) — head:
                                 lr        rf       xgb
feature                                                
FreqGoProvider             0.172977  0.014643  0.255532
Deaf                       0.041109  0.002282  0.055716
MedConditions_Diabetes     0.149365  0.006012  0.074218
MedConditions_HighBP       0.258893  0.013687  0.313645
MedConditions_LungDisease  0.002699  0.004575  0.068493
MedConditions_Depression   0.070492  0.014259  0.112413
AverageSleepNight          0.069883  0.014701  0.159286
AverageTimeSitting         0.122603  0.010116  0.302672
EverHadCancer              0.112702  0.006342  0.180847
Age                        0.859877  0.067954  0.967464

Model weights used (normalized, aligned to columns):
lr     0.3518
rf     0.3274
xgb    0.3208
dtype: float64

Top features by CompositeImportance:
                                    lr       rf      xgb  CompositeImportance
smokeStat_Never                0.17248  0.19645  0.157

In [ ]:
# ============================================================
# Multi-model SHAP + Composite Importance (magnitude + direction)
# Models: LR, RF, XGB, MLP, TabNet, TabPFN
# Handles TabPFN CPU >1000 samples safely (policy below)
# Requires your pre-split objects in memory:
# X_train_res_scaled, X_val_scaled, X_test_scaled,
# X_train, X_train_res, X_val, X_test,
# y_train, y_train_res, y_val, y_test, features
# ============================================================

import warnings, os, datetime
warnings.filterwarnings("ignore")

# --- TabPFN CPU policy: what to do if CPU & n_samples > 1000 ---
# "allow"  -> set TABPFN_ALLOW_CPU_LARGE_DATASET=1 and proceed
# "subset" -> train TabPFN on a random subset of size 1000
# "skip"   -> skip TabPFN model
TABPFN_POLICY = "allow"   # change to "subset" or "skip" if you prefer

# Optional: silence tqdm's notebook warnings
os.environ.setdefault("TQDM_DISABLE", "1")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from xgboost import XGBClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from tabpfn import TabPFNClassifier
import torch, shap
from scipy.stats import spearmanr

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
RANDOM_STATE = 42
rng = np.random.default_rng(RANDOM_STATE)

# ---------- Config knobs ----------
WEIGHT_BY = "roc_auc"        # "roc_auc" or "pr_auc"
USE_CALIBRATION = False      # multiply weights by (1 - ECE)
USE_STABILITY = False        # bootstrap stability into weights (runtime heavy)
N_BOOT = 20
TOPN_PLOT = 20
OUTROOT = "op/1_data_explore3/exports_preproc_dataset"
TIMESTAMP = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
OUTDIR = os.path.join(OUTROOT, f"composite_{TIMESTAMP}")
os.makedirs(OUTDIR, exist_ok=True)

# SHAP sampling sizes
TREE_BG_MAX = 200
TREE_EV_MAX = 300
LIN_BG_MAX  = 200
LIN_EV_MAX  = 300
KERNEL_BG_MAX = 120
KERNEL_EV_MAX = 150

# ---------- Helpers ----------
def to_df(X, columns):
    if isinstance(X, pd.DataFrame): return X.copy()
    X = np.asarray(X)
    if columns is None: columns = [f"f{i}" for i in range(X.shape[1])]
    return pd.DataFrame(X, columns=columns)

def to_series(y, name="target"):
    if isinstance(y, pd.Series): return y.copy()
    return pd.Series(np.asarray(y).ravel(), name=name)

def expected_calibration_error(y_true, y_prob, n_bins=15):
    bins = np.linspace(0, 1, n_bins + 1)
    inds = np.digitize(y_prob, bins) - 1
    ece = 0.0; N = len(y_true)
    for b in range(n_bins):
        mask = inds == b
        if mask.sum() == 0: continue
        conf = y_prob[mask].mean(); acc = y_true[mask].mean()
        ece += (mask.sum()/N) * abs(acc - conf)
    return float(ece)

def perf_summary(y_true, y_prob):
    return {"roc_auc": roc_auc_score(y_true, y_prob),
            "pr_auc":  average_precision_score(y_true, y_prob),
            "ece":     expected_calibration_error(y_true, y_prob, n_bins=15)}

def mean_abs_shap(sv, feature_names):  return pd.Series(np.abs(np.asarray(sv)).mean(axis=0), index=feature_names)
def mean_signed_shap(sv, feature_names): return pd.Series(np.asarray(sv).mean(axis=0), index=feature_names)

def _pos_index_from_model(model):
    try:
        classes = getattr(model, "classes_", None)
        if classes is not None:
            classes = np.asarray(classes)
            if 1 in classes: return int(np.where(classes == 1)[0][0])
            return int(np.argmax(classes))
    except Exception: pass
    return 1

def tree_shap(model, X_bg, X_eval, feature_names, signed=False, pos_idx=None):
    if pos_idx is None: pos_idx = _pos_index_from_model(model)
    explainer = shap.TreeExplainer(model, data=X_bg)
    sv = explainer.shap_values(X_eval)
    if isinstance(sv, list): sv = sv[pos_idx if len(sv)>pos_idx else -1]
    sv = np.asarray(sv)
    if sv.ndim == 3: sv = sv[..., pos_idx if sv.shape[-1]>pos_idx else -1]
    return (mean_signed_shap if signed else mean_abs_shap)(sv, feature_names)

def linear_shap_lr(lr, X_bg_scaled, X_eval_scaled, feature_names, signed=False):
    try:
        explainer = shap.LinearExplainer(lr, masker=shap.maskers.Independent(X_bg_scaled))
        sv = explainer.shap_values(X_eval_scaled)
    except TypeError:
        explainer = shap.LinearExplainer(lr, X_bg_scaled)
        sv = explainer.shap_values(X_eval_scaled)
    return (mean_signed_shap if signed else mean_abs_shap)(sv, feature_names)

def kernel_shap_from_proba(predict_proba_fn, X_bg_np, X_eval_np, feature_names, signed=False):
    explainer = shap.KernelExplainer(lambda X: predict_proba_fn(X)[:, 1], X_bg_np)
    sv = explainer.shap_values(X_eval_np, nsamples="auto")
    return (mean_signed_shap if signed else mean_abs_shap)(np.asarray(sv), feature_names)

def normalize_importance(s, how="sum1"):
    x = s.astype(float)
    if how == "sum1":
        x = x.clip(lower=0); denom = x.sum()
        return (x/denom) if denom>0 else x*0.0
    elif how == "minmax":
        lo, hi = x.min(), x.max()
        return (x-lo)/(hi-lo) if hi>lo else x*0.0
    else:
        raise ValueError("how must be 'sum1' or 'minmax'")

def _flatten_model_columns(df):
    if isinstance(df.columns, pd.MultiIndex):
        return df.set_axis(df.columns.get_level_values(0), axis=1)
    return df

def composite_from_models(imp_mag_dict, weights, scale="sum1", signed_dict=None):
    df_mag = _flatten_model_columns(pd.concat(imp_mag_dict, axis=1))
    for m in df_mag.columns: df_mag[m] = normalize_importance(df_mag[m], how=scale)
    w_raw = pd.Series(weights, dtype=float)
    w_norm = (w_raw / w_raw.sum()) if w_raw.sum()>0 else pd.Series(1/len(w_raw), index=w_raw.index)
    w_aligned = w_norm.reindex(df_mag.columns).fillna(0.0)
    comp_mag = df_mag.mul(w_aligned, axis=1).sum(axis=1)

    out = df_mag.copy(); out["CompositeImportance"] = comp_mag
    if signed_dict:
        df_signed = _flatten_model_columns(pd.concat(signed_dict, axis=1))
        w_signed = w_norm.reindex(df_signed.columns).fillna(0.0)
        comp_signed = df_signed.mul(w_signed, axis=1).sum(axis=1)
        out["CompositeSigned"] = comp_signed
        out["CompositeSign"] = np.sign(comp_signed).replace({0.0: 0.0})
    else:
        out["CompositeSigned"] = np.nan; out["CompositeSign"] = np.nan

    return out.sort_values("CompositeImportance", ascending=False), w_aligned

def sample_idx(n, k): return rng.choice(n, size=min(k, n), replace=False)

def plot_topn(series, title, outpath, topn=20):
    s = series.sort_values(ascending=False).head(topn)[::-1]
    plt.figure(figsize=(8, max(4, topn*0.35))); plt.barh(s.index, s.values)
    plt.title(title); plt.tight_layout(); plt.savefig(outpath, dpi=150); plt.close()

# ---------- Bring in your data ----------
X_train_res_scaled = to_df(X_train_res_scaled, features)
X_val_scaled       = to_df(X_val_scaled,       features)
X_test_scaled      = to_df(X_test_scaled,      features)
X_train     = to_df(X_train,     features)
X_train_res = to_df(X_train_res, features)
X_val       = to_df(X_val,       features)
X_test      = to_df(X_test,      features)
y_train     = to_series(y_train)
y_train_res = to_series(y_train_res)
y_val       = to_series(y_val)
y_test      = to_series(y_test)
feature_names = list(X_train.columns)

# ---------- Train models ----------
models = {}
lr = LogisticRegression(max_iter=500, class_weight="balanced", random_state=RANDOM_STATE)
lr.fit(X_train_res_scaled, y_train_res); models["lr"] = lr

rf = RandomForestClassifier(n_estimators=600, max_depth=None, max_features="sqrt",
                            class_weight="balanced", n_jobs=-1, random_state=RANDOM_STATE)
rf.fit(X_train_res, y_train_res); models["rf"] = rf

xgb = XGBClassifier(n_estimators=700, learning_rate=0.05, max_depth=5,
                    subsample=0.9, colsample_bytree=0.8, reg_lambda=1.0,
                    objective="binary:logistic", eval_metric="logloss",
                    random_state=RANDOM_STATE, n_jobs=-1, tree_method="hist")
xgb.fit(X_train_res, y_train_res); models["xgb"] = xgb

mlp = MLPClassifier(hidden_layer_sizes=(128,64), activation="relu", solver="adam",
                    alpha=1e-4, learning_rate_init=1e-3, max_iter=200,
                    early_stopping=True, random_state=RANDOM_STATE)
mlp.fit(X_train_res_scaled, y_train_res); models["mlp"] = mlp

tn = TabNetClassifier(seed=RANDOM_STATE, verbose=0, device_name=DEVICE)
tn.fit(X_train_res.values, y_train_res.values,
       eval_set=[(X_val.values, y_val.values)], eval_name=["val"],
       patience=10, max_epochs=50, batch_size=1024, virtual_batch_size=128)
models["tabnet"] = tn

# --- TabPFN safe fit (handles CPU >1000) ---
def fit_tabpfn_safe(X_np, y_np, policy="allow"):
    n = X_np.shape[0]
    if torch.cuda.is_available():
        clf = TabPFNClassifier(device="cuda", random_state=RANDOM_STATE)
        clf.fit(X_np, y_np); return clf
    # CPU path
    if n > 1000:
        if policy == "allow":
            os.environ["TABPFN_ALLOW_CPU_LARGE_DATASET"] = "1"
            clf = TabPFNClassifier(device="cpu", random_state=RANDOM_STATE)
            clf.fit(X_np, y_np); return clf
        elif policy == "subset":
            idx = rng.choice(n, size=1000, replace=False)
            clf = TabPFNClassifier(device="cpu", random_state=RANDOM_STATE)
            clf.fit(X_np[idx], y_np[idx]); return clf
        elif policy == "skip":
            return None
        else:
            raise ValueError("TABPFN_POLICY must be 'allow', 'subset', or 'skip'")
    # n <= 1000 -> safe on CPU
    clf = TabPFNClassifier(device="cpu", random_state=RANDOM_STATE)
    clf.fit(X_np, y_np); return clf

tpf = fit_tabpfn_safe(X_train_res.values, y_train_res.values, policy=TABPFN_POLICY)
if tpf is not None: models["tabpfn"] = tpf
else: print("[TabPFN] Skipped per policy")

# ---------- Validation performance (weights) ----------
def proba(model, X):
    if isinstance(model, TabNetClassifier) or isinstance(model, TabPFNClassifier):
        return model.predict_proba(X.values if isinstance(X, pd.DataFrame) else X)
    return model.predict_proba(X)

perf = {
    "lr":     perf_summary(y_val.values, proba(models["lr"],     X_val_scaled)[:,1]),
    "rf":     perf_summary(y_val.values, proba(models["rf"],     X_val)[:,1]),
    "xgb":    perf_summary(y_val.values, proba(models["xgb"],    X_val)[:,1]),
    "mlp":    perf_summary(y_val.values, proba(models["mlp"],    X_val_scaled)[:,1]),
    "tabnet": perf_summary(y_val.values, proba(models["tabnet"], X_val)[:,1]),
}
if "tabpfn" in models:
    perf["tabpfn"] = perf_summary(y_val.values, proba(models["tabpfn"], X_val)[:,1])

perf_df = pd.DataFrame(perf).T
print("Validation performance (weights source):")
print(perf_df[["roc_auc","pr_auc","ece"]].round(4))

# ---------- SHAP (magnitude + signed) ----------
bg_idx_lin  = sample_idx(len(X_train_res_scaled), LIN_BG_MAX)
ev_idx_lin  = sample_idx(len(X_val_scaled),      LIN_EV_MAX)
bg_idx_tree = sample_idx(len(X_train_res), TREE_BG_MAX)
ev_idx_tree = sample_idx(len(X_val),      TREE_EV_MAX)

imp_mag, imp_signed = {}, {}

# LR (scaled)
imp_mag["lr"]    = linear_shap_lr(lr, X_train_res_scaled.iloc[bg_idx_lin], X_val_scaled.iloc[ev_idx_lin], feature_names, signed=False)
imp_signed["lr"] = linear_shap_lr(lr, X_train_res_scaled.iloc[bg_idx_lin], X_val_scaled.iloc[ev_idx_lin], feature_names, signed=True)

# RF, XGB (unscaled)
imp_mag["rf"]    = tree_shap(rf,  X_train_res.iloc[bg_idx_tree], X_val.iloc[ev_idx_tree], feature_names, signed=False)
imp_signed["rf"] = tree_shap(rf,  X_train_res.iloc[bg_idx_tree], X_val.iloc[ev_idx_tree], feature_names, signed=True)
imp_mag["xgb"]    = tree_shap(xgb, X_train_res.iloc[bg_idx_tree], X_val.iloc[ev_idx_tree], feature_names, signed=False)
imp_signed["xgb"] = tree_shap(xgb, X_train_res.iloc[bg_idx_tree], X_val.iloc[ev_idx_tree], feature_names, signed=True)

# MLP (scaled) -> Kernel SHAP
bg_idx_ker = sample_idx(len(X_train_res_scaled), KERNEL_BG_MAX)
ev_idx_ker = sample_idx(len(X_val_scaled),      KERNEL_EV_MAX)
X_bg_mlp = X_train_res_scaled.iloc[bg_idx_ker].values
X_ev_mlp = X_val_scaled.iloc[ev_idx_ker].values
imp_mag["mlp"]    = kernel_shap_from_proba(lambda Z: mlp.predict_proba(Z), X_bg_mlp, X_ev_mlp, feature_names, signed=False)
imp_signed["mlp"] = kernel_shap_from_proba(lambda Z: mlp.predict_proba(Z), X_bg_mlp, X_ev_mlp, feature_names, signed=True)

# TabNet (unscaled) -> Kernel SHAP
X_bg_tn = X_train_res.iloc[sample_idx(len(X_train_res), KERNEL_BG_MAX)].values
X_ev_tn = X_val.iloc[sample_idx(len(X_val), KERNEL_EV_MAX)].values
imp_mag["tabnet"]    = kernel_shap_from_proba(lambda Z: tn.predict_proba(Z), X_bg_tn, X_ev_tn, feature_names, signed=False)
imp_signed["tabnet"] = kernel_shap_from_proba(lambda Z: tn.predict_proba(Z), X_bg_tn, X_ev_tn, feature_names, signed=True)

# TabPFN (unscaled) -> Kernel SHAP (only if trained)
if "tabpfn" in models:
    X_bg_tp = X_train_res.iloc[sample_idx(len(X_train_res), KERNEL_BG_MAX)].values
    X_ev_tp = X_val.iloc[sample_idx(len(X_val), KERNEL_EV_MAX)].values
    imp_mag["tabpfn"]    = kernel_shap_from_proba(lambda Z: models["tabpfn"].predict_proba(Z), X_bg_tp, X_ev_tp, feature_names, signed=False)
    imp_signed["tabpfn"] = kernel_shap_from_proba(lambda Z: models["tabpfn"].predict_proba(Z), X_bg_tp, X_ev_tp, feature_names, signed=True)

# ---------- Optional stability (off by default) ----------
stability = {}
if USE_STABILITY:
    print("Computing stability weights (bootstrap)...")
    stability["lr"]     = 1.0  # (skip bootstrap for brevity or implement like earlier)
    # You can paste the earlier bootstrap function calls here if needed.

# ---------- Build weights ----------
weights = {}
for m, met in perf.items():
    w = max(met[WEIGHT_BY], 1e-8)
    if USE_CALIBRATION: w *= (1.0 - met["ece"])
    if USE_STABILITY and m in stability: w *= max(stability[m], 1e-6)
    weights[m] = w

w_series = pd.Series(weights, name="weight"); w_series = w_series / w_series.sum()
print("\nModel weights used (normalized):"); print(w_series.round(4))

# ---------- Composite ----------
comp_df, w_used = composite_from_models(imp_mag, weights=w_series.to_dict(), scale="sum1", signed_dict=imp_signed)
print("\nTop features by CompositeImportance:"); print(comp_df.head(20).round(5))

# ---------- Test metrics ----------
def proba_np(model, X):
    if isinstance(model, TabNetClassifier) or isinstance(model, TabPFNClassifier):
        return model.predict_proba(X.values if isinstance(X, pd.DataFrame) else X)
    return model.predict_proba(X)

test_metrics = {
    "lr":     perf_summary(y_test.values, proba_np(lr,     X_test_scaled)[:,1]),
    "rf":     perf_summary(y_test.values, proba_np(rf,     X_test)[:,1]),
    "xgb":    perf_summary(y_test.values, proba_np(xgb,    X_test)[:,1]),
    "mlp":    perf_summary(y_test.values, proba_np(mlp,    X_test_scaled)[:,1]),
    "tabnet": perf_summary(y_test.values, proba_np(tn,     X_test)[:,1]),
}
if "tabpfn" in models:
    test_metrics["tabpfn"] = perf_summary(y_test.values, proba_np(models["tabpfn"], X_test)[:,1])

test_df = pd.DataFrame(test_metrics).T[["roc_auc","pr_auc","ece"]]
print("\nTest-set metrics:"); print(test_df.round(4))

# ---------- Exports ----------
pd.DataFrame(imp_mag).to_csv(os.path.join(OUTDIR, "per_model_mean_abs_shap.csv"))
pd.DataFrame(imp_signed).to_csv(os.path.join(OUTDIR, "per_model_mean_signed_shap.csv"))
comp_df.to_csv(os.path.join(OUTDIR, "composite_importance.csv"))
perf_df.to_csv(os.path.join(OUTDIR, "validation_metrics_for_weights.csv"))
w_series.to_csv(os.path.join(OUTDIR, "model_weights.csv"))
test_df.to_csv(os.path.join(OUTDIR, "test_metrics.csv"))

def plot_topn(series, title, outpath, topn=20):
    s = series.sort_values(ascending=False).head(topn)[::-1]
    plt.figure(figsize=(8, max(4, topn*0.35))); plt.barh(s.index, s.values)
    plt.title(title); plt.tight_layout(); plt.savefig(outpath, dpi=150); plt.close()

plot_topn(comp_df["CompositeImportance"], "Top Composite Importance (magnitude)",
          os.path.join(OUTDIR, "top_composite_importance.png"), topn=TOPN_PLOT)
plot_topn(comp_df["CompositeSigned"].abs(), "Top Composite |Signed| (magnitude only)",
          os.path.join(OUTDIR, "top_composite_signed_abs.png"), topn=TOPN_PLOT)

print(f"\nArtifacts saved to: {OUTDIR}")
